# Adding Google ASR output via Label Studio API

> "Because their import sucks"

- hidden: true
- branch: master
- categories: [labelstudio, googleasr, hsi]

In [1]:
def slurpfile(filename) -> str:
    with open(filename) as inf:
        return inf.read().strip()

In [2]:
host = "http://130.237.3.107:8080/api/"
api_token = slurpfile("label_studio_google")
input_dir = "/Users/joregan/Playing/hsi_google/"

In [3]:
import requests
import json
from pathlib import Path

headers = {
    "Authorization": f"Token {api_token}"
}

In [4]:
def get_projects():
    req = requests.get(f"{host}projects", headers=headers)
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [5]:
def get_project_id_from_name(name):
    projects = get_projects()
    for res in projects["results"]:
        if res["title"].strip() == name.strip():
            return res["id"]

In [6]:
get_project_id_from_name("Main 5")

6

In [7]:
def get_tasks(projectid):
    req = requests.get(f"{host}tasks", headers=headers, params={"project": projectid})
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [8]:
def index_task_filestem_to_id(tasks_data):
    tasks = tasks_data["tasks"]
    mapping = {}
    for task in tasks:
        task_id = task["id"]
        if "storage_filename" in task:
            task_raw_path = task["storage_filename"]
        else:
            task_raw_path = task["data"]["audio"]
        if not task_raw_path:
            continue
        task_stem = task_raw_path.split("/")[-1]
        mapping[task_stem] = task_id
    return mapping

In [28]:
get_project_id_from_name("Main 4")

5

In [26]:
tasks = get_tasks(5)

In [29]:
mapping = index_task_filestem_to_id(tasks)

In [30]:
mapping

{'hsi_4_0717_209_001_main.wav': 71,
 'hsi_4_0717_209_002_main.wav': 72,
 'hsi_4_0717_209_003_main.wav': 73,
 'hsi_4_0717_210_001_main.wav': 74,
 'hsi_4_0717_210_003_main.wav': 75,
 'hsi_4_0717_211_001_main.wav': 76,
 'hsi_4_0717_211_002_main.wav': 77,
 'hsi_4_0717_211_003_main.wav': 78,
 'hsi_4_0717_222_002_main.wav': 79,
 'hsi_4_0717_222_003_main.wav': 80,
 'hsi_4_0717_227_002_main.wav': 81,
 'hsi_4_0717_227_003_main.wav': 82,
 'hsi_4_0717_227_004_main.wav': 83}

In [24]:
import json

def convert_json(filename):
    segments = []
    last_end = 0.0

    with open(filename) as inf:
        data = json.load(inf)
    assert "results" in data, "no 'results' list, this is maybe not from Google ASR"
    for result in data["results"]:
        if len(result["alternatives"]) != 1:
            print("More than one alternative", result["alternatives"])
        item = result["alternatives"][0]
        if not "transcript" in item:
            continue
        text = item["transcript"]
        end_time = None
        start_time = last_end
        if "words" in item:
            if "startTime" in item["words"][0]:
                start_time = item["words"][0]["startTime"]
            if "words" in item:
                if "endTime" in item["words"][-1]:
                    end_time = item["words"][-1]["endTime"]
        else:
            if "resultEndTime" in item:
                end_time = item["resultEndTime"]
        if not end_time:
            print("Still no end time?", item)
            continue
        if start_time.endswith("s"):
            start_time = start_time[:-1]
        start_time = float(start_time)
        if end_time.endswith("s"):
            end_time = end_time[:-1]
        end_time = float(end_time)
        rec = {
            "value": {
                "start": start_time,
                "end": end_time,
                "channel": 0,
                "text": [text.strip()]
            },
            "from_name": "transcription",
            "to_name": "audio",
            "type": "textarea",
        }
        segments.append(rec)
        last_end = end_time

    return segments

In [13]:
def post_results(id, task, project, results):
    ep = f"{host}annotations/{id}/?taskID={task}&project={project}"

    cur_headers = {i: headers[i] for i in headers}
    cur_headers["Content-type"] = "application/json"

    content = {
        "was_cancelled": False,
        "ground_truth": False,
        "project": project,
        "draft_id": 0,
        "parent_prediction": None,
        "parent_annotation": None,
        "result": results
    }
    r = requests.patch(ep, data=json.dumps(content), headers=cur_headers)
    return r

In [129]:
file = f"{input_dir}hsi_3_0715_209_006_main.json"
data = convert_json(file)

In [130]:
r = post_results(98, 22, 1, data)
print(r.text)

{"id":98,"result":[{"value":{"start":9.994,"end":10.839,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":9.994,"end":10.839,"channel":0,"text":["Comprehensible."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":57.443,"end":58.064,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":57.443,"end":58.064,"channel":0,"text":["Yeah, further."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":58.264,"end":58.444,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":58.264,"end":58.444,"channel":0,"text":["Yeah."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":73.675,"end":74.576,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":73.675,"end":74.576,"channel":0,"text":

In [37]:
map2 = {k[1]: (k[0]+250) for k in enumerate(mapping)}

In [36]:
mapping

{'hsi_4_0717_209_001_main.wav': 71,
 'hsi_4_0717_209_002_main.wav': 72,
 'hsi_4_0717_209_003_main.wav': 73,
 'hsi_4_0717_210_001_main.wav': 74,
 'hsi_4_0717_210_003_main.wav': 75,
 'hsi_4_0717_211_001_main.wav': 76,
 'hsi_4_0717_211_002_main.wav': 77,
 'hsi_4_0717_211_003_main.wav': 78,
 'hsi_4_0717_222_002_main.wav': 79,
 'hsi_4_0717_222_003_main.wav': 80,
 'hsi_4_0717_227_002_main.wav': 81,
 'hsi_4_0717_227_003_main.wav': 82,
 'hsi_4_0717_227_004_main.wav': 83}

In [38]:

for task in mapping:
    jsonfile = task.replace(".wav", ".json")
    file = Path(input_dir) / jsonfile
    print(file)
    if not file.exists():
        continue
    if not task in map2:
        continue
    data = convert_json(str(file))
    r = post_results(map2[task], mapping[task], 6, data)
    # count += 1
    print(r.text)

/Users/joregan/Playing/hsi_google/hsi_4_0717_209_001_main.json
{"id":250,"result":[{"value":{"start":9.2,"end":9.7,"channel":0,"text":["Okay."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":22.6,"end":22.9,"channel":0,"text":["Yeah."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":24.7,"end":26.8,"channel":0,"text":["Now you're in inside the table, though."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":27.4,"end":28.0,"channel":0,"text":["That's better."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":30.4,"end":31.9,"channel":0,"text":["So you cuz you don't see the room."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":36.5,"end":37.2,"channel":0,"text":["This is funny."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":37.9,"end":39.4,"channel":0,"text":["Okay, sho

In [136]:
tmap = {}
count = 99
for task in mapping:
    tmap[task] = count
    count += 1


In [20]:
from pathlib import Path

for file in Path(input_dir).glob("*.json"):
    wavfile = file.stem + ".wav"
    if wavfile in mapping:
        print(wavfile, mapping[wavfile])

hsi_5_0718_210_002_main.wav 88
hsi_5_0718_210_003_main.wav 89
hsi_5_0718_211_003_main.wav 91
hsi_5_0718_222_003_main.wav 93
hsi_5_0718_210_001_main.wav 87
hsi_5_0718_211_002_main.wav 90
